# The analyze phase:
In this phase, if there are any segment files in the *probe* phase, they are merged into *whole* files. Then,  ensemble, ensemble-averaged, and space files are created from whole files.

### To-do list:

- [x] analyzing *bug* *segement* and *whole* files in both project in a serial manner.
- [ ] analyzing *bug* *segement* and *whole* files in both project in a parallel manner with Dask: memory linkage problem
- [ ] analyzing *all* *segement* and *whole* files in both project in a serial manner.
- [ ] analyzing *all* *segement* and *whole* files in both project in a parallel manner with Dask

### Naming convention:

This is the pattern of file or directory names:

1. **whole** files: whole-group-property_[-measure][-stage][.ext]
2. **ensemble** files: ensemble-group-property_[-measure][-stage][.ext]
3. **ensemble_long** files: ensemble_long-group-property_[-measure][-stage][.ext]
4. **space** files: space-group-property_[-measure][-stage][.ext]
5. **all in one** files: space-group-**species**-**allInOne**-property-_[-measure][-stage][.ext]

[keyword] means that the keyword in the file name is option. [-measure] is a physical measurement such as the auto correlation function (AFC) done on the physical 'property_'.

### Settings for testing and running on a PC

In [1]:
from glob import glob
import pathlib, re
from polyphys.manage import organizer
from polyphys.manage.parser import SumRule, TransFoci
from polyphys.analyze import analyzer
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from dask.distributed import Client
from dask import delayed
from dask import compute
client = Client(n_workers=4)
client

## PC: parallel scheme with Dask and serial scheme

Files generated in the *probe* phase can be in *segment* or *whole* version. The analyze phase can be started from the *segmens* or *wholes* depending on the type of simulation.

### files in *segment* lineage/format

#### bug

##### Sum-rule project

###### Serial with For-loop

In [ ]:
%%time
# 10 mins on MacBook Mini with 7000 lags
# 17 mins on MacBook Mini with 7000 lags
inupt_databases = glob("/Users/amirhsi_mini/research_data/probe/N*-probe-segment")
geometry = 'biaxial'
tseries_properties_bug = [
    # property_, species, group
    ('fsdT', 'Mon', 'bug'),
    ('gyrT', 'Mon', 'bug'),
    ('rfloryT', 'Mon', 'bug'),
    ('shapeT', 'Mon', 'bug'),
    ('asphericityT', 'Mon', 'bug')
]
acf_tseries_properties_bug = [
    # property_, species, group
    ('fsdT', 'Mon', 'bug'),
    ('gyrT', 'Mon', 'bug'),
    ('rfloryT', 'Mon', 'bug'),
    ('shapeT', 'Mon', 'bug'),
    ('asphericityT', 'Mon', 'bug')
]

#hist_properties_bug = [
    # direction, species, group
#    ('rflory', 'Mon', 'bug')
#]
for input_database in inupt_databases:
    print(input_database)
    analyzer.analyze_bug(
        input_database,
        '/N*/N*',
        SumRule,
        geometry,
        True,
        #nonscalar_hist_properties=nonscalar_hist_properties,
        #tseries_properties=tseries_properties_bug,
        acf_tseries_properties=acf_tseries_properties_bug,
        #hist_properties=hist_properties_bug,
        #,
        nlags=20000
    )

###### Dask: this has problem with memeory linkage

In [ ]:
inupt_databases = glob("/Users/amirhsi_mini/research_data/probe/N*-probe-segment")
geometry = 'biaxial'
analyses = []
tseries_properties_bug = [
    # property_, species, group
    ('fsdT', 'Mon', 'bug'),
    ('gyrT', 'Mon', 'bug'),
    ('rfloryT', 'Mon', 'bug'),
    ('shapeT', 'Mon', 'bug'),
    ('asphericityT', 'Mon', 'bug')
]
acf_tseries_properties_bug = [
    # property_, species, group
#    ('fsdT', 'Mon', 'bug'),
#    ('gyrT', 'Mon', 'bug'),
#    ('rfloryT', 'Mon', 'bug'),
#    ('shapeT', 'Mon', 'bug'),
    ('asphericityT', 'Mon', 'bug')
]

#hist_properties_bug = [
    # direction, species, group
#    ('rflory', 'Mon', 'bug')
#]
for input_database in inupt_databases:
    print(input_database)
    analyze_delayed = delayed(analyzer.analyze_bug)(
        input_database,
        '/N*/N*',
        SumRule,
        geometry,
        True,
        #nonscalar_hist_properties=nonscalar_hist_properties,
        #tseries_properties=tseries_properties_bug,
        acf_tseries_properties=acf_tseries_properties_bug,
        #hist_properties=hist_properties_bug,
        #nlags=100000
    )
    analyses.append(analyze_delayed)

In [ ]:
%%time
# it takes 9min and 34s.
results = compute(analyses)

##### Trans-foci project

### files in *whole* lineage/format

#### bug

##### Sum-rule project

##### Trans-Foci project

In [2]:
%%time
# Wall time: 6min 20s
geometry = 'biaxial'
probe_db = "/Users/amirhsi_mini/research_data/probe/"
project_space_pattern = "ns*nl*al*D*ac*-probe-*"
space_dbs = glob(probe_db + project_space_pattern)

nonscalar_hist_t_properties = [
    # property_, species, group, avg_axis
    ('bondsHistT', 'Foci', 'bug', 0),
    ('clustersHistT', 'Foci', 'bug', 0)#,
]

nonscalar_mat_t_properties = [
    # property_, species, group, avg_axis
    ('distMatT', 'Foci', 'bug', 0),
]


acf_tseries_properties_bug = [
    # property_, species, group
    ('fsdT', 'Mon', 'bug'),
    ('gyrT', 'Mon', 'bug'),
    ('shapeT', 'Mon', 'bug'),
    ('asphericityT', 'Mon', 'bug')
]

for space_db in space_dbs:
    analyzer.analyze_bug(
        space_db,
        '/eps*/eps*',
        TransFoci,
        geometry,
        False,
        nonscalar_hist_t_properties=nonscalar_hist_t_properties,
        nonscalar_mat_t_properties=nonscalar_mat_t_properties,
        acf_tseries_properties=acf_tseries_properties_bug,
        #hist_properties=hist_properties_bug,
        #nlags=100000
    )

[Errno 17] File exists: '/Users/amirhsi_mini/research_data/analysis/ns400nl5al1D20ac1-bug-ens'
Files are saved/overwritten in an existing directory.
[Errno 17] File exists: '/Users/amirhsi_mini/research_data/analysis/ns400nl5al1D20ac1-bug-ensAvg'
Files are saved/overwritten in an existing directory.
'['segment', 'segment_id']' are not among columns.
[Errno 17] File exists: '/Users/amirhsi_mini/research_data/analysis/ns400nl5al5D20ac1-bug-ens'
Files are saved/overwritten in an existing directory.
[Errno 17] File exists: '/Users/amirhsi_mini/research_data/analysis/ns400nl5al5D20ac1-bug-ensAvg'
Files are saved/overwritten in an existing directory.
[Errno 17] File exists: '/Users/amirhsi_mini/research_data/analysis/ns400nl5al3D20ac1-bug-ens'
Files are saved/overwritten in an existing directory.
[Errno 17] File exists: '/Users/amirhsi_mini/research_data/analysis/ns400nl5al3D20ac1-bug-ensAvg'
Files are saved/overwritten in an existing directory.
'['segment', 'segment_id']' are not among colu